In [1]:
# import os

# now_dir = os.path.dirname(os.path.abspath(__file__))
# cycle_dir = os.path.join(now_dir, "./json_cycle_finale.json")

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, explode, col

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

conf = SparkConf().setAppName("demo")
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

df = spark.read.option("multiline", "true").json("file:///home/kjh/data/bike/json_cycle_finale.json") \
                      .withColumn("rows", explode(expr("response.body.items.item"))) \
                      .select("rows.*")

# df 가 지정되어 있지 않기 때문에 df["schema"] 사용 불가
# expr 함수를 사용해 스키마를 지정

for column_name in df.columns:
    df = df.withColumn(column_name, col(column_name).cast("double"))

df = df.na.drop(subset=["avgM15Te", "avgRhm", "maxInsWsWd", "avgTs", "avgTa"])
df.count()
df.show()

assembler = VectorAssembler(inputCols=["avgM15Te", "avgRhm", "maxInsWsWd", "avgTs", "avgTa"], outputCol="features_vector")
df_assembled = assembler.transform(df)
df_assembled.show()

rf_regressor = RandomForestRegressor(labelCol="avgTa", featuresCol="features_vector", numTrees=3)
model = rf_regressor.fit(df_assembled)

predictions = model.transform(df_assembled)
predictions.select("prediction").show()

evaluator = RegressionEvaluator(labelCol="avgTa", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(f"Mean Squared Error (RMSE): {rmse}")

131072x1 화면 크기가 잘못됐습니다. 문제가 예상됩니다
23/12/13 23:40:12 WARN Utils: Your hostname, KJH-DESKTOP resolves to a loopback address: 127.0.1.1; using 192.168.69.220 instead (on interface eth0)
23/12/13 23:40:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/13 23:40:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/13 23:40:18 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+---------+---------+--------+-------+--------+--------+--------+--------+--------+------+------+-----+------+-----+------+-----+-----+-----+------+----------+-----+---------+----------+--------------+--------+------------+----------+----+--------+------------+----------+------+---------+-----+---------+-----+-----+---------+-------+---------+-------------+------+---------+------+----------+-----+---------+-----+-----+-----+-----+-----+-----------+---------+------+--------+-----+--------+--------+-------+----+
|avgCm10Te|avgCm20Te|avgCm30Te|avgCm5Te|avgLmac|avgM05Te|avgM10Te|avgM15Te|avgM30Te|avgM50Te| avgPa| avgPs|avgPv|avgRhm|avgTa|avgTca|avgTd|avgTs|avgWs|ddMefs|ddMefsHrmt|ddMes|ddMesHrmt|hr1MaxIcsr|hr1MaxIcsrHrmt|hr1MaxRn|hr1MaxRnHrmt|hr24SumRws|iscs|maxInsWs|maxInsWsHrmt|maxInsWsWd| maxPs|maxPsHrmt|maxTa|maxTaHrmt|maxWd|maxWs|maxWsHrmt|maxWsWd|mi10MaxRn|mi10MaxRnHrmt| minPs|minPsHrmt|minRhm|minRhmHrmt|minTa|minTaHrmt|minTg|n99Rn|ssDur|stnId|stnNm|sumDpthFhsc|sumFogDur|su

+---------+---------+---------+--------+-------+--------+--------+--------+--------+--------+------+------+-----+------+-----+------+-----+-----+-----+------+----------+-----+---------+----------+--------------+--------+------------+----------+----+--------+------------+----------+------+---------+-----+---------+-----+-----+---------+-------+---------+-------------+------+---------+------+----------+-----+---------+-----+-----+-----+-----+-----+-----------+---------+------+--------+-----+--------+--------+-------+----+--------------------+
|avgCm10Te|avgCm20Te|avgCm30Te|avgCm5Te|avgLmac|avgM05Te|avgM10Te|avgM15Te|avgM30Te|avgM50Te| avgPa| avgPs|avgPv|avgRhm|avgTa|avgTca|avgTd|avgTs|avgWs|ddMefs|ddMefsHrmt|ddMes|ddMesHrmt|hr1MaxIcsr|hr1MaxIcsrHrmt|hr1MaxRn|hr1MaxRnHrmt|hr24SumRws|iscs|maxInsWs|maxInsWsHrmt|maxInsWsWd| maxPs|maxPsHrmt|maxTa|maxTaHrmt|maxWd|maxWs|maxWsHrmt|maxWsWd|mi10MaxRn|mi10MaxRnHrmt| minPs|minPsHrmt|minRhm|minRhmHrmt|minTa|minTaHrmt|minTg|n99Rn|ssDur|stnId|stnNm|sum

+-------------------+
|         prediction|
+-------------------+
|  -2.83509485094851|
| 0.3150081041806392|
| 0.8628810935825858|
| 0.8628810935825858|
| 0.8628810935825858|
|  3.370464453386988|
|    6.7404660112705|
|  4.761109950664439|
|-1.0878249863163658|
|-0.6876644152150728|
| 0.8628810935825858|
|-0.6876644152150728|
|-2.2438320630984485|
|-2.2438320630984485|
|-2.2438320630984485|
|-2.2438320630984485|
|-2.2438320630984485|
| 0.8628810935825858|
|  1.435168675281932|
| 0.8628810935825858|
+-------------------+
only showing top 20 rows

Mean Squared Error (RMSE): 1.2999736172972671
